In [1]:
from pint.models import get_model
from pint.simulation import make_fake_toas_uniform, make_fake_toas_fromtim
from pint.logging import setup as setup_log
from pint.fitter import Fitter
from pint.utils import dmwavex_setup

from io import StringIO
import numpy as np
import astropy.units as u
from copy import deepcopy
from tqdm import tqdm
from matplotlib import pyplot as plt

In [2]:
setup_log(level="WARNING")

1

In [3]:
par_sim = """
    PSR           SIM2
    RAJ           05:00:00     1
    DECJ          15:00:00     1
    PEPOCH        55000
    F0            100          1
    F1            -1e-15       1 
    PHOFF         0            1
    DM            15
    TNDMAMP       -13.5
    TNDMGAM       4
    TNDMC         30
    TZRMJD        55000
    TZRFRQ        1400 
    TZRSITE       gbt
    UNITS         TDB
    EPHEM         DE440
    CLOCK         TT(BIPM2019)
"""

In [4]:
m = get_model(StringIO(par_sim))

In [5]:
ntoas = 2000
toaerrs = np.random.uniform(0.5, 2.0, ntoas) * u.us
freqs = np.linspace(500, 1500, 8) * u.MHz

In [6]:
t = make_fake_toas_uniform(
    startMJD=53001,
    endMJD=57001,
    ntoas=ntoas,
    model=m,
    freq=freqs,
    obs='gbt',
    error=toaerrs,
    add_noise=True,
    add_correlated_noise=True,
    multi_freqs_in_epoch=True,
    name='fake',
    include_bipm=True,
    include_gps=True,
)

WARNING  (pint.logging                  ): /home/abhimanyu/miniconda3/envs/pint-noise/lib/python3.10/site-packages/erfa/core.py:154 ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"


In [7]:
m.write_parfile("simulations/sim4.par")
t.write_TOA_file("simulations/sim4.tim")

In [8]:
m1 = deepcopy(m)
m1.remove_component("PLDMNoise")

Tspan = t.get_mjds().max() - t.get_mjds().min()
dmwavex_setup(m1, Tspan, n_freqs=45)

m1.write_parfile("simulations/sim4.wx.par")

WARNING  (pint.logging                  ): /home/abhimanyu/Work/PINT/src/pint/models/dmwavex.py:335 UserWarning: Frequency resolution is greater than 1/yr
